# Project 2

In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
credit_df = pd.read_csv('Credit Banking.csv')
credit_df.head()

,Credit_card,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,GTIN,MPN,Merchant_name,M_ID,Payment Method,Transaction ID,Return_ind,Return_date
0,4595.0,534,SHOES,New,RKFCM,"$1,484.74","$1,471.74",BN710,26/08/2014,17:16:17,3109172,547836726,Peninsula Beauty,YE5901,Prepaid card,30462,0,NaN
1,6679.0,396,LUGGAGE,New,TNVON,"$1,452.50","$1,442.50",KF815,26/01/2014,15:40:52,7583605,279617132,DODOcase,UW7354,Mobile carrier Billing,37927,0,NaN
2,7075.0,449,SHOES,Refurbished,VKKEA,"$4,308.02","$4,284.02",UG960,03/01/2014,1:29:05,9985192,633765867,Staples,VN6656,Credit card,80640,0,NaN
3,1418.0,145,BEDDING,New,PIUEM,"$3,272.59","$3,249.59",DK471,30/05/2014,9:10:00,7720605,734500382,Road Runner Sports,MS6579,Prepaid card,58692,0,NaN
4,6740.0,844,ELECTRONICS,Used,FMBZC,$628.67,$608.67,OJ311,14/08/2014,5:31:30,8367528,241944637,L'Occitane,GN3944,Debit card,21505,0,NaN


In [3]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Credit_card     9994 non-null   float64
 1   Product_ID      9999 non-null   int64  
 2   P_CATEGORY      9999 non-null   object 
 3   CONDTION        9999 non-null   object 
 4   Brand           9999 non-null   object 
 5   Price           9999 non-null   object 
 6   Selling_price   9999 non-null   object 
 7   Coupon_ID       9979 non-null   object 
 8   Date            9999 non-null   object 
 9   Time            9999 non-null   object 
 10  GTIN            9999 non-null   int64  
 11  MPN             9999 non-null   int64  
 12  Merchant_name   9999 non-null   object 
 13  M_ID            9999 non-null   object 
 14  Payment Method  9999 non-null   object 
 15  Transaction ID  9999 non-null   int64  
 16  Return_ind      9999 non-null   int64  
 17  Return_date     1532 non-null   o

In [4]:
credit_df.describe()

,Credit_card,Product_ID,GTIN,MPN,Transaction ID,Return_ind
count,9994.000000,9999.000000,9.999000e+03,9.999000e+03,9999.000000,9999.000000
mean,5728.449470,568.450645,5.459239e+06,5.545180e+08,49977.002100,0.153215
std,2619.421223,253.915314,2.580324e+06,2.585180e+08,28933.097165,0.360213
min,1093.000000,111.000000,1.000909e+06,1.003140e+08,198.000000,0.000000
25%,3500.000000,343.000000,3.176675e+06,3.312793e+08,24419.000000,0.000000
50%,5850.000000,607.000000,5.541936e+06,5.599136e+08,50394.000000,0.000000
75%,7979.000000,772.000000,7.601389e+06,7.765952e+08,74134.000000,0.000000
max,9950.000000,996.000000,9.997547e+06,9.997557e+08,99995.000000,1.000000


In [5]:
credit_df.isna().sum()

Credit_card          5
Product_ID           0
P_CATEGORY           0
CONDTION             0
Brand                0
Price                0
Selling_price        0
Coupon_ID           20
Date                 0
Time                 0
GTIN                 0
MPN                  0
Merchant_name        0
M_ID                 0
Payment Method       0
Transaction ID       0
Return_ind           0
Return_date       8467
dtype: int64

# Provide a meaningful treatment where the Credit Card entries are blank

In [6]:
credit_df['Credit_card'].fillna(99999, inplace=True)
credit_df.isna().sum()

Credit_card          0
Product_ID           0
P_CATEGORY           0
CONDTION             0
Brand                0
Price                0
Selling_price        0
Coupon_ID           20
Date                 0
Time                 0
GTIN                 0
MPN                  0
Merchant_name        0
M_ID                 0
Payment Method       0
Transaction ID       0
Return_ind           0
Return_date       8467
dtype: int64

In [7]:
credit_df.columns = credit_df.columns.to_series().apply(lambda x: x.strip())

# Identity where Price is equal to Selling Price even after having a Coupon Code, apply an automatic discount of 5% for those entries

In [8]:
credit_df['Price'] = credit_df['Price'].str.replace(r'\(|\)|,|\$', '', regex=True)
credit_df['Selling_price'] = credit_df['Selling_price'].str.replace(r'\(|\)|,|\$', '', regex=True)
credit_df['Price'] = credit_df['Price'].astype(float)
credit_df['Selling_price'] = credit_df['Selling_price'].astype(float)

In [9]:
credit_df['Price'] = pd.to_numeric(credit_df['Price'], errors='coerce')
credit_df['Selling_price'] = pd.to_numeric(credit_df['Selling_price'], errors='coerce')
mask = (credit_df['Price'] == credit_df['Selling_price']) & credit_df['Coupon_ID'].notna()
credit_df.loc[mask, 'Selling_price'] *= 0.95

# Make sure that the return date is after the Purchase Date

In [10]:
credit_df['Date'] = pd.to_datetime(credit_df['Date'])
credit_df['Return_date'] = pd.to_datetime(credit_df['Return_date'])
rows_to_update = credit_df['Return_date'].isnull() | (credit_df['Return_date'] < credit_df['Date'])
credit_df.loc[rows_to_update, 'Return_date'] = credit_df.loc[rows_to_update, 'Date'] + pd.DateOffset(days=10)
credit_df['Return_date'] = credit_df['Return_date'].dt.strftime('%Y-%m-%d')
credit_df

C:\Users\pavan\AppData\Local\Temp\ipykernel_15676\533133762.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  credit_df['Date'] = pd.to_datetime(credit_df['Date'])


,Credit_card,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,GTIN,MPN,Merchant_name,M_ID,Payment Method,Transaction ID,Return_ind,Return_date
0,4595.0,534,SHOES,New,RKFCM,1484.74,1471.7400,BN710,2014-08-26,17:16:17,3109172,547836726,Peninsula Beauty,YE5901,Prepaid card,30462,0,2014-09-05
1,6679.0,396,LUGGAGE,New,TNVON,1452.50,1442.5000,KF815,2014-01-26,15:40:52,7583605,279617132,DODOcase,UW7354,Mobile carrier Billing,37927,0,2014-02-05
2,7075.0,449,SHOES,Refurbished,VKKEA,4308.02,4284.0200,UG960,2014-03-01,1:29:05,9985192,633765867,Staples,VN6656,Credit card,80640,0,2014-03-11
3,1418.0,145,BEDDING,New,PIUEM,3272.59,3249.5900,DK471,2014-05-30,9:10:00,7720605,734500382,Road Runner Sports,MS6579,Prepaid card,58692,0,2014-06-09
4,6740.0,844,ELECTRONICS,Used,FMBZC,628.67,608.6700,OJ311,2014-08-14,5:31:30,8367528,241944637,L'Occitane,GN3944,Debit card,21505,0,2014-08-24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,8609.0,470,APPLIANCES,New,TTNTN,1187.85,1128.4575,FL494,2014-09-08,18:09:03,7482917,337083364,Cole Hardware,EX7729,Credit card,17306,0,2014-09-18
9995,9725.0,406,CLOTHING,Used,ONKHS,2202.20,2092.0900,DP660,2014-02-11,12:31:20,6749808,354170741,Cole Hardware,EX7730,Prepaid card,41829,0,2014-02-21
9996,1548.0,694,ELECTRONICS,New,AGSDS,3123.80,3079.8000,VI354,2014-09-02,20:06:24,7644170,101023606,Barnes & Noble,EY1659,Mobile carrier Billing,34577,0,2014-09-12
9997,4878.0,552,SHOES,New,RKFCM,1257.44,1227.4400,UQ360,2014-01-17,12:14:24,8336204,320092897,Gymboree,ZF8986,Mobile carrier Billing,43458,0,2014-01-27


# If the Coupon ID is NULL, make sure that NO discount is given, the Selling Price should be equal to Price

In [11]:
credit_df['Price'] = pd.to_numeric(credit_df['Price'], errors='coerce')
credit_df['Selling_price'] = pd.to_numeric(credit_df['Selling_price'], errors='coerce')
mask = credit_df['Coupon_ID'].isnull()
credit_df.loc[mask, 'Selling_price'] = credit_df.loc[mask, 'Price']

# Age should be greater than 18 for all the CC holders.

In [12]:
customer_df = pd.read_csv('Customer_info.csv')

mean_age_adults = customer_df[customer_df['Age'] >= 18]['Age'].mean()
customer_df.loc[customer_df['Age'] < 18, 'Age'] = mean_age_adults

customer_df

,C_ID,Email,Name,Mobile_number,Gender,Age,City,State,Address
0,3768,eddie@yahoomail.com,EDDIE,9045258449,M,83.0,Louisville,Kentucky,Masked
1,4852,rose@hotmail.com,ROSE,8834789103,F,87.0,Kansas City,Missouri,Masked
2,1174,amy@yahoomail.com,AMY,9557690013,F,31.0,Seattle,Washington,Masked
3,4807,clarence@gmail.com,CLARENCE,9394398429,M,37.0,Seattle,Washington,Masked
4,9131,johnny@hotmail.com,JOHNNY,9976623538,F,80.0,Columbus,Ohio,Masked
...,...,...,...,...,...,...,...,...,...
193,4575,tina@yahoomail.com,TINA,8819353220,F,56.0,Austin,Texas,Masked
194,3958,gordon@hotmail.com,GORDON,8853153218,M,87.0,Seattle,Washington,Masked
195,8609,jennifer@hotmail.com,JENNIFER,9935042642,F,93.0,Seattle,Washington,Masked
196,4542,michelle@hotmail.com,MICHELLE,9730382145,F,60.0,Louisville,Kentucky,Masked


In [13]:
credit_df.rename(columns = {'Credit_card':'C_ID'}, inplace = True)
credit_df.head()

,C_ID,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,GTIN,MPN,Merchant_name,M_ID,Payment Method,Transaction ID,Return_ind,Return_date
0,4595.0,534,SHOES,New,RKFCM,1484.74,1471.74,BN710,2014-08-26,17:16:17,3109172,547836726,Peninsula Beauty,YE5901,Prepaid card,30462,0,2014-09-05
1,6679.0,396,LUGGAGE,New,TNVON,1452.50,1442.50,KF815,2014-01-26,15:40:52,7583605,279617132,DODOcase,UW7354,Mobile carrier Billing,37927,0,2014-02-05
2,7075.0,449,SHOES,Refurbished,VKKEA,4308.02,4284.02,UG960,2014-03-01,1:29:05,9985192,633765867,Staples,VN6656,Credit card,80640,0,2014-03-11
3,1418.0,145,BEDDING,New,PIUEM,3272.59,3249.59,DK471,2014-05-30,9:10:00,7720605,734500382,Road Runner Sports,MS6579,Prepaid card,58692,0,2014-06-09
4,6740.0,844,ELECTRONICS,Used,FMBZC,628.67,608.67,OJ311,2014-08-14,5:31:30,8367528,241944637,L'Occitane,GN3944,Debit card,21505,0,2014-08-24


In [14]:
merged_df = pd.merge(credit_df, customer_df[['C_ID', 'Age', 'Gender', 'State']], on='C_ID', how='left')
merged_df.head()

,C_ID,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,...,MPN,Merchant_name,M_ID,Payment Method,Transaction ID,Return_ind,Return_date,Age,Gender,State
0,4595.0,534,SHOES,New,RKFCM,1484.74,1471.74,BN710,2014-08-26,17:16:17,...,547836726,Peninsula Beauty,YE5901,Prepaid card,30462,0,2014-09-05,68.0,M,Massachusetts
1,6679.0,396,LUGGAGE,New,TNVON,1452.50,1442.50,KF815,2014-01-26,15:40:52,...,279617132,DODOcase,UW7354,Mobile carrier Billing,37927,0,2014-02-05,56.0,F,Illinois
2,7075.0,449,SHOES,Refurbished,VKKEA,4308.02,4284.02,UG960,2014-03-01,1:29:05,...,633765867,Staples,VN6656,Credit card,80640,0,2014-03-11,77.0,M,Illinois
3,1418.0,145,BEDDING,New,PIUEM,3272.59,3249.59,DK471,2014-05-30,9:10:00,...,734500382,Road Runner Sports,MS6579,Prepaid card,58692,0,2014-06-09,35.0,M,Kentucky
4,6740.0,844,ELECTRONICS,Used,FMBZC,628.67,608.67,OJ311,2014-08-14,5:31:30,...,241944637,L'Occitane,GN3944,Debit card,21505,0,2014-08-24,55.0,F,Texas


# Customer Segmentation based on Spend in Dollars, based on Swipes, segmentation 

In [15]:
def age_category(Age,Gender):
    
    '''
    Age group categories:
    
    Young Females    :18<=age<=35
    Mid age Females  :36<=age<=50
    Old Females      :age>50
    Young Males      :18<=age<=35
    Mid age Males    :36<=age<=50
    Old Males        :age>50
    
    '''
    if Age>=18 and Age<=35 and Gender=='F':
        return 'Young Females'
    elif Age>=36 and Age<=50 and Gender=='F':
        return 'Mid-age Females'
    elif Age>50 and Gender=='F':
        return 'Old Females'
    elif Age>=18 and Age<=35 and Gender=='M':
        return 'Young Males'
    elif Age>=36 and Age<=50 and Gender=='M':
        return 'Mid-age Males'
    elif Age>50 and Gender=='M':
        return 'Old Males'
    else:
        return 'Unknown'
merged_df['Customer_Segment'] = merged_df.apply(lambda row: age_category(row['Age'], row['Gender']), axis=1)
merged_df.head()

,C_ID,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,...,Merchant_name,M_ID,Payment Method,Transaction ID,Return_ind,Return_date,Age,Gender,State,Customer_Segment
0,4595.0,534,SHOES,New,RKFCM,1484.74,1471.74,BN710,2014-08-26,17:16:17,...,Peninsula Beauty,YE5901,Prepaid card,30462,0,2014-09-05,68.0,M,Massachusetts,Old Males
1,6679.0,396,LUGGAGE,New,TNVON,1452.50,1442.50,KF815,2014-01-26,15:40:52,...,DODOcase,UW7354,Mobile carrier Billing,37927,0,2014-02-05,56.0,F,Illinois,Old Females
2,7075.0,449,SHOES,Refurbished,VKKEA,4308.02,4284.02,UG960,2014-03-01,1:29:05,...,Staples,VN6656,Credit card,80640,0,2014-03-11,77.0,M,Illinois,Old Males
3,1418.0,145,BEDDING,New,PIUEM,3272.59,3249.59,DK471,2014-05-30,9:10:00,...,Road Runner Sports,MS6579,Prepaid card,58692,0,2014-06-09,35.0,M,Kentucky,Young Males
4,6740.0,844,ELECTRONICS,Used,FMBZC,628.67,608.67,OJ311,2014-08-14,5:31:30,...,L'Occitane,GN3944,Debit card,21505,0,2014-08-24,55.0,F,Texas,Old Females


In [16]:
customer_segment_spend = merged_df.groupby('Customer_Segment')['Selling_price'].sum()
customer_segment_spend = customer_segment_spend.reset_index(name='Total Spending')
customer_segment_spend
#print(merged_df['Customer_Segment'].value_counts())  


,Customer_Segment,Total Spending
0,Mid-age Females,1.079320e+06
1,Mid-age Males,6.522626e+05
2,Old Females,1.028062e+07
3,Old Males,8.452362e+06
4,Unknown,8.556770e+03
5,Young Females,2.646789e+06
6,Young Males,3.138927e+06


In [17]:
customer_segment_Payment_Method = merged_df.groupby(['Customer_Segment'])['Payment Method'].value_counts()
customer_segment_Payment_Method = customer_segment_Payment_Method.reset_index(name='count')
customer_segment_Payment_Method

,Customer_Segment,Payment Method,count
0,Mid-age Females,Mobile carrier Billing,174
1,Mid-age Females,Credit card,105
2,Mid-age Females,Prepaid card,91
3,Mid-age Females,Gift card,21
4,Mid-age Females,Paypal wallet,7
5,Mid-age Males,Credit card,105
6,Mid-age Males,Mobile carrier Billing,91
7,Mid-age Males,Prepaid card,62
8,Mid-age Males,Debit card,7
9,Mid-age Males,Deirect debits,7


# Calculate the spend in terms of Product, State and Payment method

In [18]:
spend_by_product = merged_df.groupby('P_CATEGORY')['Selling_price'].sum()
print("Spend by Product:")
spend_by_product = spend_by_product.reset_index(name='Total Spending')
spend_by_product

Spend by Product:


,P_CATEGORY,Total Spending
0,APPLIANCES,1.509852e+06
1,BABY CLOTHING,1.341636e+06
2,BABY TOYS,1.113453e+06
3,BEDDING,1.660588e+06
4,CLOTHING,1.971971e+06
5,COMPUTERS,3.060817e+06
6,DECOR,3.199401e+06
7,ELECTRONICS,2.683993e+06
8,GAMES,1.515089e+06
9,KITCHEN & DINING,2.017056e+06


In [19]:
spend_by_state = merged_df.groupby('State')['Selling_price'].sum()
print("Spend by State:")
spend_by_state = spend_by_state.reset_index(name='Total Spending')
spend_by_state

Spend by State:


,State,Total Spending
0,Arizona,1.605134e+06
1,California,7.589398e+06
2,Illinois,2.006161e+06
3,Kentucky,2.179944e+06
4,Massachusetts,1.743463e+06
5,Missouri,7.790177e+05
6,Nevada,1.689787e+06
7,Ohio,1.026222e+06
8,Texas,5.509295e+06
9,Washington,2.121857e+06


In [20]:
spend_by_payment_method = merged_df.groupby('Payment Method')['Selling_price'].sum()
print("Spend by Payment Method:")
spend_by_payment_method = spend_by_payment_method.reset_index(name='Total Spending')
spend_by_payment_method

Spend by Payment Method:


,Payment Method,Total Spending
0,Credit card,9.522582e+06
1,Debit card,8.307461e+05
2,Deirect debits,1.535727e+05
3,Gift card,1.820680e+05
4,Mobile carrier Billing,1.084784e+07
5,Paypal wallet,2.236580e+05
6,Prepaid card,4.498367e+06


# Calculate the highest 5 spending in terms of Product, State and Payment method

In [21]:
top_product_spending = merged_df.groupby('P_CATEGORY')['Selling_price'].sum().nlargest(5)
print("Top 5 spending in Product category:")
top_product_spending = top_product_spending.reset_index(name='Total Spending')
top_product_spending

Top 5 spending in Product category:


,P_CATEGORY,Total Spending
0,DECOR,3.199401e+06
1,COMPUTERS,3.060817e+06
2,ELECTRONICS,2.683993e+06
3,OFFICE SUPPLIES,2.465481e+06
4,SHOES,2.296989e+06


In [22]:
top_state_spending = merged_df.groupby('State')['Selling_price'].sum().nlargest(5)
print("Top 5 spending in State category:")
top_state_spending = top_state_spending.reset_index(name='Total Spending')
top_state_spending

Top 5 spending in State category:


,State,Total Spending
0,California,7.589398e+06
1,Texas,5.509295e+06
2,Kentucky,2.179944e+06
3,Washington,2.121857e+06
4,Illinois,2.006161e+06


In [23]:
top_payment_method_spending = merged_df.groupby('Payment Method')['Selling_price'].sum().nlargest(5)
print("Top 5 spending in Payment method category:")
top_payment_method_spending=top_payment_method_spending.reset_index(name='Total Spending')
top_payment_method_spending

Top 5 spending in Payment method category:


,Payment Method,Total Spending
0,Mobile carrier Billing,1.084784e+07
1,Credit card,9.522582e+06
2,Prepaid card,4.498367e+06
3,Debit card,8.307461e+05
4,Paypal wallet,2.236580e+05


# Give opinion on return category like customers returning the products belongs to which state, age group, condition, category of the product or is it related to discount

In [24]:
def time_group(time_str):
    try:
        time_obj = pd.to_datetime(time_str)
        hour = time_obj.hour
    except (ValueError, AttributeError):
        return 'Invalid Time'
    if 3 <= hour < 5:
        return 'Early Morning'
    elif 5 <= hour < 11:
        return 'Morning'
    elif 11 <= hour < 15:
        return 'Afternoon'
    elif 15 <= hour < 19:
        return 'Evening'
    elif 19 <= hour < 23:
        return 'Night'
    else:
        return 'Midnight'

merged_df['Time_Group'] = merged_df['Time'].apply(time_group)

In [25]:
return_df = merged_df[merged_df['Return_ind'] == 1]
return_df

,C_ID,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,...,M_ID,Payment Method,Transaction ID,Return_ind,Return_date,Age,Gender,State,Customer_Segment,Time_Group
9,5974.0,470,LUGGAGE,Used,TNVON,4783.25,4764.25,ER876,2014-09-23,5:13:44,...,EI6775,Credit card,28877,1,2014-09-30,60.197861,M,Texas,Old Males,Morning
28,7979.0,389,CLOTHING,New,GTFFL,3747.69,3704.69,JL124,2014-01-17,13:11:00,...,GN4079,Credit card,88335,1,2014-01-24,20.000000,F,Missouri,Young Females,Afternoon
48,8532.0,544,APPLIANCES,Used,BUDLO,2135.85,2085.85,WR271,2014-04-27,21:07:55,...,HW9976,Mobile carrier Billing,73080,1,2014-05-04,21.000000,F,Massachusetts,Young Females,Night
73,6502.0,662,BABY CLOTHING,New,CTAOI,4658.93,4609.93,JJ698,2014-03-10,21:17:54,...,YE5901,Mobile carrier Billing,10537,1,2014-10-10,27.000000,F,Texas,Young Females,Night
101,4139.0,534,DECOR,Refurbished,ZACKV,4981.73,4942.73,FU981,2014-09-29,21:56:13,...,UO9656,Mobile carrier Billing,40788,1,2014-10-06,86.000000,M,Nevada,Old Males,Night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,2297.0,875,DECOR,Refurbished,LBVIW,4993.38,4982.38,GJ767,2014-11-16,13:34:42,...,EX7730,Mobile carrier Billing,81742,1,2014-11-23,70.000000,F,Washington,Old Females,Afternoon
2076,2804.0,572,LUGGAGE,New,CENQJ,3277.74,3236.74,RQ273,2014-04-01,23:34:49,...,ZF8986,Credit card,53335,1,2014-04-11,87.000000,M,Texas,Old Males,Midnight
2077,6071.0,539,SHOES,Refurbished,RKFCM,2837.17,2788.17,SP318,2014-06-03,19:56:24,...,EY1658,Mobile carrier Billing,40144,1,2014-06-13,60.197861,M,Arizona,Old Males,Night
2078,6286.0,773,SHOES,New,MQSLF,645.10,611.10,NN757,2014-09-07,8:17:34,...,RM9837,Mobile carrier Billing,68552,1,2014-09-17,63.000000,M,Texas,Old Males,Morning


In [26]:
return_analysis = return_df.groupby(['State', 'Age', 'Time', 'CONDTION', 'P_CATEGORY', 'Coupon_ID','Time_Group']).size().reset_index(name='Return_Count')
print("Return Category Analysis:")
return_analysis

Return Category Analysis:


,State,Age,Time,CONDTION,P_CATEGORY,Coupon_ID,Time_Group,Return_Count
0,Arizona,33.0,23:47:14,New,SHOES,EK545,Midnight,2
1,Arizona,33.0,3:06:59,Refurbished,OFFICE SUPPLIES,TU198,Early Morning,2
2,Arizona,33.0,7:29:57,Refurbished,LUGGAGE,XP523,Morning,2
3,Arizona,33.0,8:06:44,Refurbished,OFFICE SUPPLIES,YJ954,Morning,1
4,Arizona,41.0,16:41:57,Used,COMPUTERS,XB852,Evening,1
...,...,...,...,...,...,...,...,...
1295,Washington,87.0,9:18:48,New,DECOR,AD474,Morning,2
1296,Washington,93.0,10:17:48,New,APPLIANCES,TR607,Morning,2
1297,Washington,93.0,16:33:25,Used,BABY CLOTHING,DG998,Evening,1
1298,Washington,93.0,18:09:03,New,APPLIANCES,FL494,Evening,1


In [27]:
returns_by_state = return_df.groupby('State')['Return_ind'].value_counts()
returns_by_state=returns_by_state.reset_index(name='No.of orders')

returns_by_state

,State,Return_ind,No.of orders
0,Arizona,1,104
1,California,1,450
2,Illinois,1,122
3,Kentucky,1,118
4,Massachusetts,1,114
5,Missouri,1,52
6,Nevada,1,101
7,Ohio,1,54
8,Texas,1,297
9,Washington,1,120


In [28]:
returns_by_agegroup = return_df.groupby('Customer_Segment')['Return_ind'].value_counts()
returns_by_agegroup=returns_by_agegroup.reset_index(name='No.of orders')

returns_by_agegroup

,Customer_Segment,Return_ind,No.of orders
0,Mid-age Females,1,62
1,Mid-age Males,1,40
2,Old Females,1,596
3,Old Males,1,484
4,Young Females,1,162
5,Young Males,1,188


In [29]:

return_time = return_df.groupby('Time_Group')['Return_ind'].value_counts()
return_time=return_time.reset_index(name='No.of orders')

return_time


,Time_Group,Return_ind,No.of orders
0,Afternoon,1,282
1,Early Morning,1,113
2,Evening,1,249
3,Midnight,1,251
4,Morning,1,379
5,Night,1,258


In [30]:
return_category = return_df.groupby('P_CATEGORY')['Return_ind'].value_counts()
return_category=return_category.reset_index(name='No.of orders')

return_category

,P_CATEGORY,Return_ind,No.of orders
0,APPLIANCES,1,92
1,BABY CLOTHING,1,73
2,BABY TOYS,1,62
3,BEDDING,1,83
4,CLOTHING,1,119
5,COMPUTERS,1,171
6,DECOR,1,182
7,ELECTRONICS,1,159
8,GAMES,1,113
9,KITCHEN & DINING,1,120


In [31]:
return_condition = return_df.groupby('CONDTION')['Return_ind'].value_counts()
return_condition=return_condition.reset_index(name='No.of orders')

return_condition

,CONDTION,Return_ind,No.of orders
0,New,1,712
1,Refurbished,1,387
2,Used,1,433


# Create a profile of customers in terms of timing of their order

In [32]:
order_time = merged_df.groupby('Time_Group')['Return_ind'].value_counts()
order_time=order_time.reset_index(name='No.of orders')

order_time

,Time_Group,Return_ind,No.of orders
0,Afternoon,0,1558
1,Afternoon,1,282
2,Early Morning,0,705
3,Early Morning,1,113
4,Evening,0,1389
5,Evening,1,249
6,Midnight,0,1344
7,Midnight,1,251
8,Morning,0,2050
9,Morning,1,379


# Which payment method is providing more discount for customers

In [33]:
merged_df['discount_%']=((merged_df['Price']-merged_df['Selling_price'])/merged_df['Price'])*100
merged_df.head()

,C_ID,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,...,Payment Method,Transaction ID,Return_ind,Return_date,Age,Gender,State,Customer_Segment,Time_Group,discount_%
0,4595.0,534,SHOES,New,RKFCM,1484.74,1471.74,BN710,2014-08-26,17:16:17,...,Prepaid card,30462,0,2014-09-05,68.0,M,Massachusetts,Old Males,Evening,0.875574
1,6679.0,396,LUGGAGE,New,TNVON,1452.50,1442.50,KF815,2014-01-26,15:40:52,...,Mobile carrier Billing,37927,0,2014-02-05,56.0,F,Illinois,Old Females,Evening,0.688468
2,7075.0,449,SHOES,Refurbished,VKKEA,4308.02,4284.02,UG960,2014-03-01,1:29:05,...,Credit card,80640,0,2014-03-11,77.0,M,Illinois,Old Males,Midnight,0.557100
3,1418.0,145,BEDDING,New,PIUEM,3272.59,3249.59,DK471,2014-05-30,9:10:00,...,Prepaid card,58692,0,2014-06-09,35.0,M,Kentucky,Young Males,Morning,0.702807
4,6740.0,844,ELECTRONICS,Used,FMBZC,628.67,608.67,OJ311,2014-08-14,5:31:30,...,Debit card,21505,0,2014-08-24,55.0,F,Texas,Old Females,Morning,3.181319


In [34]:
average_discount_by_method = merged_df.groupby('Payment Method')['discount_%'].mean()
max_discount_method = average_discount_by_method.idxmax()
print("The payment method providing the highest average discount is:", max_discount_method)
average_discount_by_method = merged_df.groupby('Payment Method')['discount_%'].sum().nlargest(5)
average_discount_by_method=average_discount_by_method.reset_index(name='Sum of Discounts')
average_discount_by_method

The payment method providing the highest average discount is: Mobile carrier Billing


,Payment Method,Sum of Discounts
0,Mobile carrier Billing,13035.544619
1,Credit card,9694.448006
2,Prepaid card,3098.656108
3,Debit card,889.632428
4,Deirect debits,152.417929


# Create a profile for high value items vs low value items and relate that wrt to their number of orders

In [35]:
def price_category(Price):
    
    '''
    The items whose price less than 1000 are categorised as low price items
    The items whose price in between 1000 and 2500 are categorised as moderate price items
    The items whose price greter than 2500 are categorised as costly items2
    
    '''
    if Price>0 and Price<=1000:
        return 'low price items'
    elif Price>1000 and Price<=2500:
        return 'moderate price items'
    return 'costly items'

merged_df['price_category']=merged_df['Price'].apply(lambda x: price_category(x))

In [36]:
Items_profile=merged_df.groupby('price_category')['Customer_Segment'].value_counts()
Items_profile=Items_profile.unstack(fill_value=0)
Items_profile

Customer_Segment,Mid-age Females,Mid-age Males,Old Females,Old Males,Unknown,Young Females,Young Males
price_category,,,,,,,
costly items,251,125,2118,1733,1,559,629
low price items,70,56,733,565,2,189,279
moderate price items,77,91,1012,878,2,273,356


# Do you think if merchant provides more discount then can it will lead to increase in number of orders?

yes, we can see that most of the orders have discount. So by giving discounts orders may increase

In [37]:
def discount_percentage(discount):
    
    '''
    Age group categories:
    
    Very low discount    :0<discount<=5
    low discount         :5<discount<=25
    moderate discount    :25<discount<=65
    High discount        :discount>65
    
    '''
    if 0<discount<=5 :
        return 'Very low discount'
    elif 5<discount<=25 :
        return 'low discount'
    elif 25<discount<=65 :
        return 'moderate discount'
    elif discount>65:
        return 'High discount'
    else:
        return 'No discount'
    
merged_df['discount_wise_segment'] = merged_df.apply(lambda row: discount_percentage(row['discount_%']), axis=1)
merged_df.head()

,C_ID,Product_ID,P_CATEGORY,CONDTION,Brand,Price,Selling_price,Coupon_ID,Date,Time,...,Return_ind,Return_date,Age,Gender,State,Customer_Segment,Time_Group,discount_%,price_category,discount_wise_segment
0,4595.0,534,SHOES,New,RKFCM,1484.74,1471.74,BN710,2014-08-26,17:16:17,...,0,2014-09-05,68.0,M,Massachusetts,Old Males,Evening,0.875574,moderate price items,Very low discount
1,6679.0,396,LUGGAGE,New,TNVON,1452.50,1442.50,KF815,2014-01-26,15:40:52,...,0,2014-02-05,56.0,F,Illinois,Old Females,Evening,0.688468,moderate price items,Very low discount
2,7075.0,449,SHOES,Refurbished,VKKEA,4308.02,4284.02,UG960,2014-03-01,1:29:05,...,0,2014-03-11,77.0,M,Illinois,Old Males,Midnight,0.557100,costly items,Very low discount
3,1418.0,145,BEDDING,New,PIUEM,3272.59,3249.59,DK471,2014-05-30,9:10:00,...,0,2014-06-09,35.0,M,Kentucky,Young Males,Morning,0.702807,costly items,Very low discount
4,6740.0,844,ELECTRONICS,Used,FMBZC,628.67,608.67,OJ311,2014-08-14,5:31:30,...,0,2014-08-24,55.0,F,Texas,Old Females,Morning,3.181319,low price items,Very low discount


In [38]:
orders_in_diffrent_discount_segment=merged_df.groupby('P_CATEGORY')['discount_wise_segment'].value_counts()
orders_in_diffrent_discount_segment
orders_in_diffrent_discount_segment = orders_in_diffrent_discount_segment.unstack(fill_value=0)
orders_in_diffrent_discount_segment

discount_wise_segment,High discount,No discount,Very low discount,low discount,moderate discount
P_CATEGORY,,,,,
APPLIANCES,0,0,539,6,0
BABY CLOTHING,28,0,442,25,7
BABY TOYS,0,1,377,76,7
BEDDING,0,0,497,33,14
CLOTHING,0,2,717,90,0
COMPUTERS,0,5,1153,70,0
DECOR,21,1,976,106,28
ELECTRONICS,0,2,922,84,0
GAMES,0,1,645,47,0


In [39]:
Project2_results = {
    'customer segment spend': customer_segment_spend,
    'customer segment Payment Method': customer_segment_Payment_Method,
    'spend_by_product': spend_by_product,
    'spend_by_state': spend_by_state,
    'spend_by_payment_method': spend_by_payment_method,
    'top_product_spending': top_product_spending,
    'top_state_spending': top_state_spending,
    'top_payment_method_spending': top_payment_method_spending,
    'returns_by_state': returns_by_state,
    'returns_by_agegroup': returns_by_agegroup,
    'return_time': return_time,
    'order_time':order_time,
    'average_discount_by_method': average_discount_by_method,
    'Items_profile': Items_profile,
    'orders_in_discount_segment':orders_in_diffrent_discount_segment
}

# Create a Pandas Excel writer
writer = pd.ExcelWriter('project2_results.xlsx', engine='xlsxwriter')

# Write each result to a separate sheet in the Excel file
for task, result in Project2_results.items():
    result.to_excel(writer, sheet_name=task, index=False)

# Save the Excel file
writer.save()


C:\Users\pavan\AppData\Local\Temp\ipykernel_15676\445451434.py:27: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
